## Penjelasan

Langkah pertama yang dilakukan adalah meng-import modul pandas dan numpy serta mengambil data lapor.csv pada folder di data_path

In [38]:
import pandas as pd
import numpy as np

In [39]:
data_path = "data/data_target/"
filename = "questions.csv"

Membaca file lapor.csv dengan memanfaatkan modul Pandas dengan menggunakan fungsi .head(), maka data yang ditampilkan merupakan 5 data teratas 

In [40]:
train_df = pd.read_csv(data_path + filename, delimiter=',')
train_df.head()

,pertanyaan,Kategori
0,bagaimana kisah tentang mukasyafah nya sayyidi...,KISAH SAYYIDINA UMAR VS MALAIKAT MUNKAR-NAKIR
1,ceritakan kisah sayyidina umar vs munkar nakir,KISAH SAYYIDINA UMAR VS MALAIKAT MUNKAR-NAKIR
2,cerita sayyidina umar vs munkar nakir termaktu...,KISAH SAYYIDINA UMAR VS MALAIKAT MUNKAR-NAKIR
3,kenapa wudhu menggunakan air?,ALASAN BERSUCI DENGAN AIR
4,apakah bisa wudhu tidak menggunakan air?,ALASAN BERSUCI DENGAN AIR


menampilkan jumlah masing-masing kategori

In [41]:
kategori_total_list = train_df["Kategori"].value_counts()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (kategori_total_list)

ALASAN BERSUCI DENGAN AIR                                                  13
MAHAR NIKAH BERASAL DARI ORANG LAIN                                         5
TAFSIR MIMPI MASJID ROBOH                                                   5
TAJWID : MAD LAZIM HARFI MUSYABBA?                                          5
SHOLAT : HUKUM BACA BASMALAH SAAT AKAN SHOLAT                               4
JIKA MAKMUM KETINGGALAN BACA FATIHAH KARENA IMAM SHOLAT TERLALU CEPAT       4
HUKUM BERBUAT DOSA BAGI ANAK YANG BELUM BALIGH II                           4
HUKUM MELUDAH KE ARAH KIBLAT                                                4
HUKUM LAKI-LAKI MEMAKAI MINYAK ZA?FARON                                     4
hukum sholat qobliyah shubuh setelah shalat shubuh                          4
HUKUM JUAL BELI KUCING                                                      4
HUKUM BERQURBAN DENGAN AYAM JAGO                                            4
BOLEHKAH SUAMI MENGELUARKAN MANI SAAT BERCUMBU DENGAN ISTRI YANG

menggabungkan kategori yang memiliki jumlah berita sedikit dengan list kategori lain yang lebih umum dan memiliki jumlah yang sudah banyak.

In [43]:
kategori = train_df["Kategori"].values

Menampilkan isi pertanyaan

In [42]:
pertanyaan = train_df["pertanyaan"].values
pertanyaan

array(['bagaimana kisah tentang mukasyafah nya sayyidina ali r.a tentang sayyidina umar vs mungkar nakir',
       'ceritakan kisah sayyidina umar vs munkar nakir',
       'cerita sayyidina umar vs munkar nakir termaktub dari kitab apa?',
       'kenapa wudhu menggunakan air?',
       'apakah bisa wudhu tidak menggunakan air?',
       'kalau tidak ada air bagaimana bisa berwuduhu?', 'wudhu tanpa air',
       'mengapa wuduh harus menggunakan air?', 'haruskah wudhu pake air?',
       "kenapa sholat maghrib 3 rokaat. Ashar 4 Zhuhur 4 Subuh 2 roka'at. Kok tidak disamakan saja",
       'mengapa jumlah rakaat sholat beragam',
       'mengapa jumlah rakaat sholat berbeda-beda',
       'mengapa jumlah rakaat sholat tidak sama',
       'kenapa sholat maghrib 3 rakaat',
       'mengapa sholat dzuhur ashar dan isya 4 rakaat',
       'kenapa sholat subuh 2 rakaat',
       'bolehkah mahar nikah berasal dari orang lain?',
       'calon suami mendapat sponsor untuk dijadikan mahar nikah seperti pada a

Mendapatkan word embedding dari database glove

In [44]:
def getWordEmbedding(word, cursor):
#     word = word.replace("'", "''")
    sql = """select vec from term where term like %s"""
    cursor.execute(sql, (str(word),))
    data = cursor.fetchall()
    if len(data) > 0:
        decoded_vec = json.JSONDecoder().decode(data[0][0])
        vec = np.asarray(decoded_vec, dtype=np.float32)
        return True, vec
    else:
        return False, data
    
def myTokenizer(content, lower=True):
    raw = content.split(' ')
    remover = re.compile("[^a-zA-Z-]")
    
    token = []
    
    for i in raw:
        term = remover.sub('', i)
        if lower == True:
            term = term.lower()
        token.append(term)
#     tokenized = filter(None, token)
    
#     return tokenized
    return token

def sentenceToVec(string):

    if type(string) is not str:
        return np.zeros((300,), dtype=float)
   
    string = string.replace('\n', '')
    string = np.array(myTokenizer(string))
#     print(string)
    feature = None
    begin = True
    for word in string:
        stat, vec = getWordEmbedding(word, cursor)
        if not stat:
            continue
        if begin:
            begin = False
            feature = vec
        else:
            feature += vec
#             feature = np.concatenate([feature, vec])
#     print('running') 
    if feature is not None:
        feature = feature/np.linalg.norm(feature)
    else:
        feature = np.zeros((300,), dtype=float)
    
    return feature

import re
import json
import unicodedata
import mysql.connector
from tqdm import tqdm

db = mysql.connector.connect(user="root", password='', database="glove")
cursor = db.cursor(buffered=True)

pbar = tqdm(total=len(pertanyaan))
ftr = []
for i in pertanyaan:
    try:
        ftr.append(sentenceToVec(i))
    except Exception as err:
        print (err)
        print (i)
        break
    pbar.update(1)
pbar.close()
ftr = np.array(ftr)

100%|███████████████████████████████████████████| 222/222 [00:15<00:00, 10.41it/s]


In [45]:
nan_idx = []
for i in range(len(kategori)):
    if kategori[i] is np.nan:
        nan_idx.append(i)
kategori_c = np.delete(kategori, nan_idx)
ftr_c = np.delete(ftr, nan_idx, 0)

In [46]:
cls = []
kategori_u = pd.unique(kategori_c)
kategori_u = kategori_u.tolist()
for i in range(len(kategori_c)):
    one_hot = np.zeros((len(kategori_u),), dtype=int)
    idx = kategori_u.index(kategori_c[i])
    one_hot[idx] = 1
    cls.append(one_hot)
cls = np.array(cls)

print ("Mengecek array size dari fitur dan kelas...")
print ("size of ftr " + str(ftr_c.shape))
print ("size of cls " + str(cls.shape))

Mengecek array size dari fitur dan kelas...
size of ftr (222, 300)
size of cls (222, 73)


In [48]:
np.save(data_path + "ftr-data-target.npy", ftr_c)
np.save(data_path + "cls-data-target.npy", cls)

## Splitting Dataset
Memisahkan Source Dataset dan Target Dataset menjadi Train Set dan Test Tes

In [49]:
data_source_path = "data/data_target/"

ftr_data_target = np.load(data_source_path + "ftr-data-target.npy")
cls_data_target = np.load(data_source_path + "cls-data-target.npy")

In [51]:
data_source_path_splitted = "data/splitted/data_target/"

In [53]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=0)
for train_index, test_index in sss.split(ftr_data_target, cls_data_target):
    X, X_test = ftr_data_target[train_index], ftr_data_target[test_index]
    Y, Y_test = cls_data_target[train_index], cls_data_target[test_index]
    break
# X = np.expand_dims(X, axis=2)
# X_test = np.expand_dims(X_test, axis=2)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [34]:
np.save(data_source_path_splitted + "X.npy", X)
np.save(data_source_path_splitted + "Y.npy", Y)
np.save(data_source_path_splitted + "X_test.npy", X_test)
np.save(data_source_path_splitted + "Y_test.npy", Y_test)